In [1]:
%matplotlib inline
import numpy as np 
import pandas as pd 
import random
import pickle
import torch
from matplotlib import pyplot as plt 
import torchvision.models as models
from torchvision import transforms as T
import torchvision.transforms.functional as TF
import torch.nn as nn 
import torch.nn.functional as F
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
import cv2
import os
from pathlib import Path
from IPython.display import clear_output
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
d = {'ImageId': df['ImageId'].unique()}
train_df = pd.DataFrame(d)
train_df['ClassesList'] = ['' for i in range(len(train_df))]
train_df['EncodedPixels'] = ['' for i in range(len(train_df))]
for i in range(len(train_df)):
    train_df['ClassesList'][i] = df.loc[df['ImageId']==train_df['ImageId'][i]]['ClassId'].values.tolist()
    train_df['EncodedPixels'][i] = df.loc[df['ImageId']==train_df['ImageId'][i]]['EncodedPixels'].values.tolist()

In [3]:
def make_mask( encoded, shape=(1600,256)):
    # Делим на два списка в соответствии с кодировкой
    if isinstance(encoded, str):
        encoded = list(map(int, encoded.split(' ')))
    full,pixel,number = [],[],[]
    [pixel.append(encoded[i]) if i%2==0 else number.append(encoded[i]) for i in range(0, len(encoded))]
    # "Раскрываем" кодировку, получаем индексы закрашенных пикселей
    k=0
    for i in range(len(number)):
        for j in range(number[i]):
            ind = pixel[i]+j
            full.append(ind-1)
        k +=number[i]
    # Создаем массив под готовое изображение    
    mask = np.zeros((1600*256,1), dtype=int)
    # Закрашиваем соответствующие пиксели
    mask[full] = 255
    #преобразем к размерам фотографий металла
    res = np.reshape(mask,(1600, 256)).T
    res = Image.fromarray(res.astype(np.uint8))
    return res

In [4]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [5]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE_1 = 800
RESCALE_SIZE_2 = 128
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [6]:
class SteelDataset(Dataset):
    def __init__(self, names, df, mode):
        super().__init__()
        self.names = names
        self.mode = mode
        if self.mode != 'test':
            self.df = df

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.names)
        
    def __len__(self):
        return self.len_
      
    def load_sample(self, file, mode):
        if mode == 'test':
            image = Image.open('../input/severstal-steel-defect-detection/test_images/'+ file).convert("RGB")
        else:
            image = Image.open('../input/severstal-steel-defect-detection/train_images/'+ file).convert("RGB")
        image.load()
        return image
    
    def __getitem__(self, index):
            transforms_tens = T.Compose([
            T.ToTensor()])

            # загружаем и меняем размер изображения
            img = self._prepare_sample(self.load_sample(self.names[index], self.mode))

            if self.mode == 'test':
                img = np.array(img)
                max_value = 256 ** ((img.dtype == np.uint16) + 1) - 1
                img = (img / max_value).astype(np.float32)
                img = transforms_tens(img)
                return img
            
            # загружаем классы и маски
            labels = list(self.df['ClassesList'].loc[self.df['ImageId'] == self.names[index]])[0]
            mask = list(self.df['EncodedPixels'].loc[self.df['ImageId'] == self.names[index]])[0]
            num_obj = len(labels)
            masks = np.zeros((RESCALE_SIZE_2, RESCALE_SIZE_1, 4), dtype=np.float32)
            masks = np.transpose(masks,(2, 0, 1))
            for i in range(4):
                for j in range(num_obj):
                    if i == (labels[j]-1):
                # раскодирование масок
                        masks[i] = np.array(self._prepare_sample(make_mask(mask[j])))
                        masks[i] = np.array(masks[i])
                        masks[i] = masks[i] / 255

            # преобазование в тензоры
            masks = torch.as_tensor(masks, dtype=torch.float32)
            label = torch.as_tensor(labels)
            
            # преобразуем изображения и маски
            if self.mode == "train":
                img, masks= transforms_all(img,masks)
                
            img = np.array(img)
            max_value = 256 ** ((img.dtype == np.uint16) + 1) - 1
            img = (img / max_value).astype(np.float32)
            img = transforms_tens(img)
            
            return img, masks
        
    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE_1, RESCALE_SIZE_2))
        return image

In [7]:
def transforms_all(image, segmentation):
    if random.random() > 0.5:
        image = TF.autocontrast(image)
    if random.random() > 0.5:
        image = TF.hflip(image)
        segmentation = TF.hflip(segmentation)
    if random.random() > 0.5:
        image = TF.vflip(image)
        segmentation = TF.vflip(segmentation)
    return image, segmentation

In [8]:
from sklearn.model_selection import train_test_split
train_val_names = train_df['ImageId']
train_files,val_files = train_test_split(train_val_names, train_size=0.75)
val_dataset = SteelDataset(list(val_files), train_df,  mode='val')
train_dataset = SteelDataset(list(train_files),train_df, mode='train')

In [10]:
# k = int(np.random.uniform(0,50))
# res = train_dataset[k]

# plt.show()
# plt.figure(figsize=(13, 9))
# plt.imshow(np.rollaxis(res[0].numpy(), 0, 3), cmap='gray')
# plt.title('real')
# plt.axis('off')
# plt.show()

# plt.figure(figsize=(13, 9))
# plt.imshow(res[1].numpy())
# plt.title('True')
# plt.axis('off')
# plt.show()
# # print(res[2])

In [ ]:
train_dataset[k][1].shape

In [11]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)
    
class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.double_conv = DoubleConv(in_channels, out_channels)
        self.down_sample = nn.MaxPool2d(2)

    def forward(self, x):
        skip_out = self.double_conv(x)
        down_out = self.down_sample(skip_out)
        return (down_out, skip_out)

    
class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, up_sample_mode):
        super(UpBlock, self).__init__()
        if up_sample_mode == 'conv_transpose':
            self.up_sample = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2)        
        elif up_sample_mode == 'bilinear':
            self.up_sample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            raise ValueError("Unsupported `up_sample_mode` (can take one of `conv_transpose` or `bilinear`)")
        self.double_conv = DoubleConv(in_channels, out_channels)

    def forward(self, down_input, skip_input):
        x = self.up_sample(down_input)
        x = torch.cat([x, skip_input], dim=1)
        return self.double_conv(x)

    
class UNet(nn.Module):
    def __init__(self, out_classes=4, up_sample_mode='conv_transpose'):
        super(UNet, self).__init__()
        self.up_sample_mode = up_sample_mode
        # Downsampling Path
        self.down_conv1 = DownBlock(3, 64)
        self.down_conv2 = DownBlock(64, 128)
        self.down_conv3 = DownBlock(128, 256)
        self.down_conv4 = DownBlock(256, 512)
        # Bottleneck
        self.double_conv = DoubleConv(512, 1024)
        # Upsampling Path
        self.up_conv4 = UpBlock(512 + 1024, 512, self.up_sample_mode)
        self.up_conv3 = UpBlock(256 + 512, 256, self.up_sample_mode)
        self.up_conv2 = UpBlock(128 + 256, 128, self.up_sample_mode)
        self.up_conv1 = UpBlock(128 + 64, 64, self.up_sample_mode)
        # Final Convolution
        self.conv_last = nn.Conv2d(64, 4, kernel_size=1)

    def forward(self, x):
        x, skip1_out = self.down_conv1(x)
        x, skip2_out = self.down_conv2(x)
        x, skip3_out = self.down_conv3(x)
        x, skip4_out = self.down_conv4(x)
        x = self.double_conv(x)
        x = self.up_conv4(x, skip4_out)
        x = self.up_conv3(x, skip3_out)
        x = self.up_conv2(x, skip2_out)
        x = self.up_conv1(x, skip1_out)
        x = self.conv_last(x)
        return x

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer, batch_size):
    model.train()
    running_loss = 0.0
    dice = 0.0
    iou = 0.0
    for X_batch, Y_batch in train_loader:
        inputs = X_batch.to(DEVICE)
        labels = Y_batch.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels.long())
        loss.backward()
        optimizer.step()
        running_loss += loss.detach().cpu().numpy() 
        dice += dice_metric(outputs, labels)
        iou += mIoU(outputs, labels)
    train_loss = running_loss / len(train_loader)
    train_dice = dice / len(train_loader)
    train_IoU = iou / len(train_loader)
    return train_loss, train_dice, train_IoU

In [ ]:
def eval_epoch(model, val_loader, criterion, batch_size):
    model.eval()
    dice = 0.0
    iou = 0.0
    running_loss = 0.0
    for X_batch, Y_batch in val_loader:
        inputs = X_batch.to(DEVICE)
        labels = Y_batch.to(DEVICE)

        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs,labels.long())
            running_loss += loss.item()
            dice += dice_metric(outputs, labels)
            iou += mIoU(outputs, labels)
    val_loss = running_loss / len(val_loader)
    val_dice = dice / len(val_loader)
    val_IoU = iou / len(val_loader)
    return val_loss, val_dice, val_IoU

In [ ]:
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size,num_workers = 2, shuffle=True)#,collate_fn = collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,num_workers = 2, shuffle=False)#,collate_fn = collate_fn)
    history = []
    maxIoU = 0
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} train_dice: {t_dice:0.4f} train_IoU: {t_iou:0.4f}\
    \nValidation  val_loss: {v_loss:0.4f} val_dice: {v_dice:0.4f} val_IoU: {v_iou: 0.4f}"
    criterion = nn.BCEWithLogitsLoss()#nn.CrossEntropyLoss()
    
    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        params = model.parameters()
        opt = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(opt,
                                                   step_size=20,
                                                   gamma=0.1)
        for epoch in range(epochs):
            train_loss, train_dice, train_IoU = fit_epoch(model, train_loader, criterion, opt, batch_size)
            lr_scheduler.step()    
            val_loss, val_dice, val_IoU = eval_epoch(model, val_loader, criterion, batch_size)
            history.append((train_loss, train_dice, train_IoU, val_loss, val_dice, val_IoU))
            if val_IoU > maxIoU:
                maxIoU = val_IoU
                torch.save(model, './weight_unet.dat')
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss, t_dice = train_dice, t_iou = train_IoU, \
                                           v_loss=val_loss, v_dice = val_acc, v_iou = val_IoU ))   
    return history

In [ ]:
def compute_ious(pred, label, classes=[1], ignore_index=255, only_present=True):
    '''computes iou for one ground truth mask and predicted mask'''
    pred[label == ignore_index] = 0
    ious = []
    for c in classes:
        label_c = label == c
        if only_present and np.sum(label_c) == 0:
            ious.append(np.nan)
            continue
        pred_c = pred == c
        intersection = np.logical_and(pred_c, label_c).sum()
        union = np.logical_or(pred_c, label_c).sum()
        if union != 0:
            ious.append(intersection / union)
    return ious if ious else [1]

def mIoU(outputs, labels, classes=None):
    '''computes mean iou for a batch of ground truth masks and predicted masks'''
    ious = []
    preds = np.copy(outputs) # copy is imp
    labels = np.array(labels) # tensor to np
    for pred, label in zip(preds, labels):
        ious.append(np.nanmean(compute_ious(pred, label, classes=[1])))
    iou = np.nanmean(ious)
    return iou


In [12]:
# def mIoU(pred_mask, mask, smooth=1e-10, n_classes=4):
#     with torch.no_grad():
#         pred_mask = F.softmax(pred_mask, dim=1)
#         pred_mask = torch.argmax(pred_mask, dim=1)
#         pred_mask = pred_mask.contiguous().view(-1)
#         mask = mask.contiguous().view(-1)

#         iou_per_class = []
#         for clas in range(0, n_classes): #loop per pixel class
#             true_class = pred_mask == clas
#             true_label = mask == clas

#             if true_label.long().sum().item() == 0: #no exist label in this loop
#                 iou_per_class.append(np.nan)
#             else:
#                 intersect = torch.logical_and(true_class, true_label).sum().float().item()
#                 union = torch.logical_or(true_class, true_label).sum().float().item()

#                 iou = (intersect + smooth) / (union +smooth)
#                 iou_per_class.append(iou)
#         return np.nanmean(iou_per_class)

In [14]:
# def dice_metric1(y_pred,y_true, smooth=1):
# #     print(y_pred.shape)
#     y_pred = F.softmax(y_pred, dim=1)
#     y_pred = torch.argmax(y_pred, dim=1)
# #     y_pred_f = y_pred.contiguous().view( -1)
# #     y_true_f = y_true.contiguous().view(-1)
#     y_true_f = torch.flatten(y_true)
#     y_pred_f = torch.flatten(y_pred)
#     intersection = torch.sum(y_true_f.float() * y_pred_f.float())
#     return (2. * intersection + smooth) / (torch.sum(y_true_f) + torch.sum(y_pred_f.float()) + smooth)

In [ ]:
def dice_metric(probability, truth, threshold=0.5, reduction='none'):
    batch_size = len(truth)
    with torch.no_grad():
        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = truth.float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = dice_pos
    return dice

In [ ]:
model = UNet().to(DEVICE)

In [ ]:
# class conv_block_nested(nn.Module):
    
#     def __init__(self, in_ch, mid_ch, out_ch):
#         super(conv_block_nested, self).__init__()
#         self.activation = nn.ReLU(inplace=True)
#         self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
#         self.bn1 = nn.BatchNorm2d(mid_ch)
#         self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
#         self.bn2 = nn.BatchNorm2d(out_ch)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.activation(x)
        
#         x = self.conv2(x)
#         x = self.bn2(x)
#         output = self.activation(x)

#         return output

In [ ]:
# class NestedUNet(nn.Module):

#     def __init__(self, in_ch=3, out_ch=5):
#         super(NestedUNet, self).__init__()

#         n1 = 64
#         filters = [64,128, 256, 512, 1024]

#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

#         self.conv0_0 = conv_block_nested(in_ch, filters[0], filters[0])
#         self.conv1_0 = conv_block_nested(filters[0], filters[1], filters[1])
#         self.conv2_0 = conv_block_nested(filters[1], filters[2], filters[2])
#         self.conv3_0 = conv_block_nested(filters[2], filters[3], filters[3])
#         self.conv4_0 = conv_block_nested(filters[3], filters[4], filters[4])

#         self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0])
#         self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
#         self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
#         self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])

#         self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1], filters[0], filters[0])
#         self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2], filters[1], filters[1])
#         self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])

#         self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1], filters[0], filters[0])
#         self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])

#         self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])

#         self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)


#     def forward(self, x):
        
#         x0_0 = self.conv0_0(x)
#         x1_0 = self.conv1_0(self.pool(x0_0))
#         x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(x1_0)], 1))

#         x2_0 = self.conv2_0(self.pool(x1_0))
#         x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
#         x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(x1_1)], 1))

#         x3_0 = self.conv3_0(self.pool(x2_0))
#         x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
#         x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1)], 1))
#         x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(x1_2)], 1))

#         x4_0 = self.conv4_0(self.pool(x3_0))
#         x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
#         x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
#         x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
#         x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(x1_3)], 1))

#         output = self.final(x0_4)
#         return output


In [ ]:
# model = NestedUNet().to(DEVICE)

In [ ]:
# class UnetConv2dBlock(nn.Module):

#     def __init__(self, in_channel: int, mid_channel: int, out_channel: int, kernel_size: list = [3,3], stride_size: list = [1,1], activation: str = 'relu'):
#         super().__init__()

#         if activation == 'swish':
#             self.conv2dblock = nn.Sequential(
#             nn.Conv2d(in_channel, mid_channel, kernel_size=kernel_size[0], stride=stride_size[0], padding=1),
#             nn.BatchNorm2d(mid_channel),
#             Swish(),
#             nn.Conv2d(mid_channel, out_channel, kernel_size=kernel_size[1], stride=stride_size[1], padding=1),
#             nn.BatchNorm2d(out_channel),
#             Swish(),
#         )
#         else:
#             self.conv2dblock = nn.Sequential(
#                 nn.Conv2d(in_channel, mid_channel, kernel_size=kernel_size[0], stride=stride_size[0], padding=1),
#                 nn.BatchNorm2d(mid_channel),
#                 nn.ReLU(inplace=True),
#                 nn.Conv2d(mid_channel, out_channel, kernel_size=kernel_size[1], stride=stride_size[1], padding=1),
#                 nn.BatchNorm2d(out_channel),
#                 nn.ReLU(inplace=True)
#             )

#     def forward(self, x):
#         return self.conv2dblock(x)



# class Swish(nn.Module):

#     def __init__(self, slope = 1):
#         super().__init__()
#         self.slope = slope # * torch.nn.Parameter(torch.ones(1))

#     def forward(self, x):
#         return self.slope * x * torch.sigmoid(x)



# class DownSample(nn.Module):

#     def __init__(self):
#         super().__init__()
#         self.maxpooling = nn.MaxPool2d(2)

#     def forward(self, x):
#         return self.maxpooling(x)



# class UpSample(nn.Module):

#     def __init__(self, in_channel=None, bilinear: bool = True):
#         super().__init__()

#         if bilinear:
#             self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
#         else:
#             self.up = nn.ConvTranspose2d(in_channel , in_channel // 2, kernel_size=2, stride=2)


#     def forward(self, x1, x2):
#         x1 = self.up(x1)
        
#         diffY = x2.size()[2] - x1.size()[2]
#         diffX = x2.size()[3] - x1.size()[3]

#         x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
#                         diffY // 2, diffY - diffY // 2])
#         x = torch.cat([x2, x1], dim=1)
#         return x

In [ ]:
# class UnetPlusPlus(nn.Module):
    
#     def __init__(self, in_channel, num_classes: int, deep_supervision: bool=False):
#         super(UnetPlusPlus, self).__init__()
#         self.num_classes = num_classes
#         self.metrics = 0 
#         self.deep_supervision = deep_supervision

#         num_filter = [32, 64, 128, 256, 512]

#         self.down = DownSample()
#         self.up = UpSample()

#         self.conv_block_0_0 = UnetConv2dBlock(in_channel, num_filter[0], num_filter[0])
#         self.conv_block_1_0 = UnetConv2dBlock(num_filter[0], num_filter[1], num_filter[1])
#         self.conv_block_2_0 = UnetConv2dBlock(num_filter[1], num_filter[2], num_filter[2])
#         self.conv_block_3_0 = UnetConv2dBlock(num_filter[2], num_filter[3], num_filter[3])
        
#         self.conv_block_0_1 = UnetConv2dBlock(num_filter[0]+num_filter[1], num_filter[0], num_filter[0])
#         self.conv_block_1_1 = UnetConv2dBlock(num_filter[1]+num_filter[2], num_filter[1], num_filter[1])
#         self.conv_block_2_1 = UnetConv2dBlock(num_filter[2]+num_filter[3], num_filter[2], num_filter[2])

#         self.conv_block_0_2 = UnetConv2dBlock(2*num_filter[0]+num_filter[1], num_filter[0], num_filter[0])
#         self.conv_block_1_2 = UnetConv2dBlock(2*num_filter[1]+num_filter[2], num_filter[1], num_filter[1])

#         self.conv_block_0_3 = UnetConv2dBlock(3*num_filter[0]+num_filter[1], num_filter[0], num_filter[0])

#         if self.deep_supervision:
#             self.final1 = nn.Conv2d(num_filter[0], self.num_classes, kernel_size=1)
#             self.final2 = nn.Conv2d(num_filter[0], self.num_classes, kernel_size=1)
#             self.final3 = nn.Conv2d(num_filter[0], self.num_classes, kernel_size=1)
#         else:
#             self.final = nn.Conv2d(num_filter[0], self.num_classes, kernel_size=1)

#     def forward(self, x):

#         x0_0 = self.conv_block_0_0(x)

#         x1_0 = self.conv_block_1_0(self.down(x0_0))
#         x0_1 = self.conv_block_0_1(self.up(x1_0, x0_0))
        
#         x2_0 = self.conv_block_2_0(self.down(x1_0))
#         x1_1 = self.conv_block_1_1(self.up(x2_0, x1_0))
#         x0_2 = self.conv_block_0_2(self.up(x1_1, torch.cat([x0_0, x0_1], 1)))

#         x3_0 = self.conv_block_3_0(self.down(x2_0))
#         x2_1 = self.conv_block_2_1(self.up(x3_0, x2_0))
#         x1_2 = self.conv_block_1_2(self.up(x2_1, torch.cat([x1_0, x1_1],1)))
#         x0_3 = self.conv_block_0_3(self.up(x1_2, torch.cat([x0_0, x0_1, x0_2], 1)))

#         if self.deep_supervision:
#             output1 = self.final1(x0_1)
#             output2 = self.final2(x0_2)
#             output3 = self.final3(x0_3)
#             return [output1, output2, output3]

#         else:
#             out = self.final(x0_3)
#             return out

In [ ]:
# model = UnetPlusPlus(3,5).to(DEVICE)

In [ ]:
# history = train(train_dataset, val_dataset, model, epochs = 36, batch_size = 5)

In [ ]:
# history1 = train(train_dataset, val_dataset, model, epochs = 10, batch_size = 10)

In [ ]:
train_loss, train_acc, train_IoU, val_loss, val_acc, val_IoU = zip(*history)

In [ ]:
# train_loss1, train_acc1, train_IoU1, val_loss1, val_acc1, val_IoU1 = zip(*history1)

In [ ]:
# ttrain_loss = []
# ttrain_acc = []
# ttrain_IoU =[]
# tval_loss =[]
# tval_acc =[]
# tval_IoU=[]
# for i in range(40):
#     ttrain_loss.append(train_loss[i])
#     ttrain_acc.append(train_acc[i]) 
#     ttrain_IoU.append(train_IoU[i]) 
#     tval_loss.append(val_loss[i]) 
#     tval_acc.append(val_acc[i]) 
#     tval_IoU.append(val_IoU[i])
# for i in range(4):
#     ttrain_loss.append(train_loss1[i])
#     ttrain_acc.append(train_acc1[i]) 
#     ttrain_IoU.append(train_IoU1[i]) 
#     tval_loss.append(val_loss1[i]) 
#     tval_acc.append(val_acc1[i]) 
#     tval_IoU.append(val_IoU1[i])

In [ ]:
#     fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(30,13))
#     ax1.plot(train_loss, label='train', marker='o')
#     ax1.plot(val_loss, label='val', marker='o')
#     ax1.set_title('Loss per epoch')
#     ax1.set_ylabel('loss');
#     ax1.set_xlabel('epoch')
#     ax1.legend(), ax1.grid()
    
#     ax2.plot(train_IoU, label='train_IoU', marker='*')
#     ax2.plot(val_IoU, label='val_IoU',  marker='*')
#     ax2.set_title('Score per epoch')
#     ax2.set_ylabel('mean IoU')
#     ax2.set_xlabel('epoch')
#     ax2.legend(), ax2.grid()

#     ax3.plot(train_acc, label='train_accuracy', marker='*')
#     ax3.plot(val_acc, label='val_accuracy',  marker='*')
#     ax3.set_title('Accuracy per epoch')
#     ax3.set_ylabel('Accuracy')
#     ax3.set_xlabel('epoch')
#     ax3.legend(), ax3.grid()
# #     plt.savefig('0.710615.png')
#     plt.show()

In [70]:
def predict_image_mask_score(model, image, mask, metric = dice_metric):
    model.eval()
    
    image = image.to(DEVICE)
    mask = mask.to(DEVICE)
    with torch.no_grad():
        
        image = image.unsqueeze(0)
        mask = mask.unsqueeze(0)
        
        output = model(image)
#         print(output)
        score = metric(output, mask)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked, score

In [73]:
def score(model, dataset, metric = dice_metric):
    res_score = []
    for i in tqdm(range(len(val_dataset))):
        img, mask = dataset[i]
        pred_mask, score = predict_image_mask_score(model, img, mask, metric)
        res_score.append(score)
    return res_score

In [69]:
def dice_metric(prob, true, threshold=0.5, reduction='none'):
    '''Calculates dice of positive and negative images seperately'''
    '''probability and truth must be torch tensors'''
    batch_size = len(true)
    prob = F.softmax(prob, dim=1)
    prob = torch.argmax(prob, dim=1)
#     print(prob)
#     print(prob.shape)
    probability = np.zeros((RESCALE_SIZE_2, RESCALE_SIZE_1, 4), dtype=np.float32)
    probability = np.transpose(probability,(2, 0, 1))
    truth = np.zeros((RESCALE_SIZE_2, RESCALE_SIZE_1, 4), dtype=np.float32)
    truth = np.transpose(truth,(2, 0, 1))
    for i in range(4):
        for j in range(RESCALE_SIZE_2):
            for k in range(RESCALE_SIZE_1):
#                 print(prob[j][k])
                if i == prob[0][j][k]-1:
                    probability[i][j][k] = 1
                if i == true[0][j][k]-1:
                    truth[i][j][k] = 1
    probability = torch.as_tensor(probability) 
    truth = torch.as_tensor(truth)
    with torch.no_grad():
        probability = probability.contiguous().view(batch_size, -1)
        truth = truth.contiguous().view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = probability
        t = truth 

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = dice_pos

    return dice

In [17]:
model = torch.load('../input/unet-weight/weight_unet_wew.dat')

In [74]:
print(np.mean(score(model, val_dataset)))

In [71]:
k1,k2 = int(np.random.uniform(0,100)),int(np.random.uniform(0,100))
image1, mask1 = val_dataset[k1]
pred_mask1, score1 = predict_image_mask_score(model, image1, mask1)

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,13))
ax1.imshow(np.rollaxis(image1.numpy(), 0, 3))
ax1.set_title('Picture');
ax1.set_axis_off()

ax2.imshow(mask1)
ax2.set_title('Ground truth')
ax2.set_axis_off()

ax3.imshow(pred_mask1)
ax3.set_title('UNet | '+ str(score1))
ax3.set_axis_off()
print(score1)
image2, mask2 = val_dataset[k2]
pred_mask2, score2 = predict_image_mask_score(model, image2, mask2)

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(25,13))
ax1.imshow(np.rollaxis(image2.numpy(), 0, 3))
ax1.set_title('Picture');
ax1.set_axis_off()

ax2.imshow(mask2)
ax2.set_title('Ground truth')
ax2.set_axis_off()

ax3.imshow(pred_mask2)
ax3.set_title('UNet | '+str(score2))
ax3.set_axis_off()